## Treliça

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import solve
%matplotlib inline

In [2]:
A_0 = 100  # mm^2
E = 2e15   # N/mm^2
L = 1.5e3*np.array([1,np.sqrt(2),1])     # mm
F = np.array([0,500,-500]).reshape(3,1)  # N
r_v = 1
r_A = [1,1,1]

In [3]:
def k_bar(Ai,Li):
    return(Ai * E / Li)

def k_matrix(A):
    A1,A2,A3 = A[0],A[1],A[2]
    L1,L2,L3 = L[0],L[1],L[2]
    k_1 = k_bar(A1,L1)
    k_2 = k_bar(A2,L2)
    k_3 = k_bar(A3,L3)
    return(np.array([k_1 + k_2,-k_1,-k_1,
                    -k_1,k_1 + k_2,k_1,
                    -k_1,k_1,k_1]).reshape(3,3))

def u_desl(A):
    K = k_matrix(A)
    u = solve(K,F)
    return(u)

def dk_dA(A):
    L1,L2,L3 = L[0],L[1],L[2]
    dk1 = E / (2 * L1) * np.array([1,-1,-1,
                                 -1,1,1,
                                 -1,1,1]).reshape(3,3)
    dk2 = E / (L2) * np.array([0,0,0,
                               0,1,0,
                               0,0,0]).reshape(3,3)
    dk3 = E / (L3) * np.array([1,0,0,
                               0,0,0,
                               0,0,0]).reshape(3,3)
    return(dk1,dk2,dk3)
    

def du_dA(A):
    u = u_desl(A)
    K = k_matrix(A)
    dk1,dk2,dk3 = dk_dA(A)
    return(solve(K,np.matmul(-dk1,u)),
           solve(K,np.matmul(-dk2,u)),
           solve(K,np.matmul(-dk3,u)))

def f(A):
    u = u_desl(A)
    return(0.5 * np.matmul(F.T,u)[0])

def g_V(A):
    return(np.dot(A,L) - 300 * np.sum(L))

def g_A(A):
    return(A_0 - np.array(A))

def lagr(A):
    return(f(A) + r_v * max(0,g_V(A))**2
           + np.dot(r_A,np.maximum(g_A(A),0)**2))

def grad_dir(A):    
    L1,L2,L3 = L[0],L[1],L[2]
    Vg = r_v * max(0,g_V(A))
    Ag = r_A*np.maximum(g_A([1,2,103]),0)
    uA = du_dA(A)
    return(np.array([L1 * Vg - Ag[0] +
           np.matmul(F.T,uA[0])[0][0],
           L2 * Vg - Ag[1] +
           np.matmul(F.T,uA[1])[0][0],
           L3 * Vg - Ag[2] +
           np.matmul(F.T,uA[2])[0][0]]).reshape(3,1))

def grad_adj(A):
    L1,L2,L3 = L[0],L[1],L[2]
    Vg = r_v * max(0,g_V(A))
    Ag = r_A*np.maximum(g_A([1,2,103]),0)
    dk1,dk2,dk3 = dk_dA(A)
    u = u_desl(A)
    ga1 = -np.matmul(np.matmul(dk1,u).T,u)[0][0]
    ga2 = -np.matmul(np.matmul(dk2,u).T,u)[0][0]
    ga3 = -np.matmul(np.matmul(dk3,u).T,u)[0][0]
    return(np.array([L1 * Vg - Ag[0] + ga1,
                     L2 * Vg - Ag[1] + ga2,
                     L3 * Vg - Ag[2] + ga3]).reshape(3,1))

In [4]:
print(grad_adj([100,100,100]),'\n',grad_dir([100,100,100]))

[[ -9.90000000e+01]
 [ -9.80000000e+01]
 [ -3.75000000e-11]] 
 [[ -9.90000000e+01]
 [ -9.80000000e+01]
 [ -3.75000000e-11]]
